In [1]:
import os
import numpy as np

from skimage.io import imread, imshow, imsave
from skimage.transform import resize
from skimage.exposure import rescale_intensity
from data_helper import load_image_paths

%matplotlib inline

In [2]:
IMAGE_FOLDER_PATH = 'dataset/frames/'

image_paths = load_image_paths(image_folder_path=IMAGE_FOLDER_PATH,
                               data_type='train',
                               label_type='obj',
                               image_type='left')

image_paths += load_image_paths(image_folder_path=IMAGE_FOLDER_PATH,
                                data_type='train',
                                label_type='obj',
                                image_type='right')

image_paths += load_image_paths(image_folder_path=IMAGE_FOLDER_PATH,
                                data_type='train',
                                label_type='obj',
                                image_type='head')

image_paths += load_image_paths(image_folder_path=IMAGE_FOLDER_PATH,
                                data_type='test',
                                label_type='obj',
                                image_type='left')

image_paths += load_image_paths(image_folder_path=IMAGE_FOLDER_PATH,
                                data_type='test',
                                label_type='obj',
                                image_type='right')

image_paths += load_image_paths(image_folder_path=IMAGE_FOLDER_PATH,
                                data_type='test',
                                label_type='obj',
                                image_type='head')

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/frames/train/lab/1/Lhand'

In [111]:
def resize_and_save_image(image_path, smallest_size=256):
    image = imread(image_path)
    
    # Resize image.
    height, width = float(image.shape[0]), float(image.shape[1])
    scale = smallest_size / width if (height > width) else smallest_size / height
    new_height = int(height*scale)
    new_width = int(width*scale)
    image = resize(image, output_shape=(new_height, new_width))
    
    # Normalize to 0-255 uint8
    image = rescale_intensity(image, out_range=(0, 255)).astype(np.uint8)
    
    # Save image
    # Create a new path for saving.
    dataset_folder_idx = [i for i, name in enumerate(image_path.split('/')) if name == 'dataset'][0]
    resized_folder_idx = dataset_folder_idx + 1
    
    f_names = image_path.split('/')
    f_names.insert(resized_folder_idx, 'resized')
    
    resized_image_path = os.path.join(*f_names)
    resized_image_folder = os.path.join(*resized_image_path.split('/')[:-1])
    
    if not os.path.exists(resized_image_folder):
        os.makedirs(resized_image_folder)
    
    imsave(resized_image_path, resized_image)